In [52]:
from unidecode import unidecode 
import pandas as pd
import re
import json
from datetime import datetime
import math

In [53]:
input_json_data = 'D:/TRUNG/Raw/MongoDB/Quảng Ninh/Mongo_QuangNinh.json'
output_json_data = "Mongo_QuangNinh_cleaned.json"
csv_data = 'D:/TRUNG/Booking/Quảng Ninh Booking/Booking_QuangNinh.csv'
output_csv_data = 'SoSanh_Hotel_QuangNinh.csv'

In [54]:
with open(input_json_data, "r", encoding='utf-8') as file:
     raw_data = file.read()
processed_data1 = re.sub(r"NumberLong\((\d+)\)", r"\1", raw_data)
processed_data2 = re.sub(r"NumberInt\((\d+)\)", r"\1", processed_data1)
processed_data3 = re.sub(r'ISODate\("(.*?)"\)', r'"\1"', processed_data2)   
data = json.loads(processed_data3)
with open(output_json_data, "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

In [55]:
extracted_data = []
for item in data:
    _id = item.get("_id")
    name = item.get("name")
    lng = item.get("location", {}).get("lng")
    lat = item.get("location", {}).get("lat")
    address_components = {comp["types"][0]: comp["searchName"] for comp in item.get("addressComponents", [])}
    housenumber = address_components.get("housenumber", None)
    street = address_components.get("street", None)
    admin_level_4 = address_components.get("admin_level_4", None)
    extracted_data.append({
        "_id": _id,
        "name": name,
        "lng": lng,
        "lat": lat,
        "housenumber_searchName": housenumber,
        "street_searchName": street,
        "admin_level_4_searchName": admin_level_4
    })
mongo_df = pd.DataFrame(extracted_data)
display(mongo_df)

,_id,name,lng,lat,housenumber_searchName,street_searchName,admin_level_4_searchName
0,5ca5cab9c59d76fe0074d864,Tuần Châu Marina- Shophouse Mặt Biển,106.980643,20.921139,None,ngoc chau,tuan chau
1,5cca5f30654fdf045cb7f663,BÊ TÔNG TRƯỜNG THÀNH,107.033591,20.985491,None,cai lan,bai chay
2,5d1c5bc6f5e0be238c616700,Cty Than Hạ Long,107.088383,20.979784,75,tran phu,cao xanh
3,5d1c5bc6f5e0be238c616702,Cty Cp Xây Dựng Đô Thh Thành phố Hạ Long,107.084953,20.954967,43,dong tam,tran hung dao
4,5d2bf1b801938510fc5e7516,CGV Vincom Plaza Hạ Long,107.083978,20.950159,None,dong ho,bach dang
...,...,...,...,...,...,...,...
97555,65a576a90429b457b6f4e32f,Quán lá Trang Đại,106.696651,20.834728,None,ngo quyen,hong thai dong
97556,6600e550bc45c11c71a36862,Trung Tâm Đăng Kiểm XCG 1406D - Quảng Ninh,107.027687,20.977842,None,cai lan,gieng day
97557,6600e550bc45c11c71a36863,Trung Tâm Đăng Kiểm XCG 1405D - Quảng Ninh,107.107045,21.000697,None,cum cong nghiep,ha khanh
97558,6600e918bc45c11c71a36953,Trung Tâm Đăng Kiểm XCG 1406D - Quảng Ninh,107.027687,20.977842,None,cai lan,gieng day


In [56]:
hotel_data = pd.read_csv(csv_data, encoding='utf-8')
display(hotel_data)

,Hotel Name,Address,Latitude,Longitude,Link
0,LAS VEGAS HOTEL,Lô 01 A6 Khu đô Thị Ao Tiên - Thôn 6 Xã Hạ Lon...,21.08612795,107.45164109,https://www.booking.com/hotel/vn/las-vegas.vi....
1,Legacy Yen Tu - MGallery,"Thuong Yen Cong Commune Quang Ninh Province, 2...",21.1563582,106.7180108,https://www.booking.com/hotel/vn/legacy-yen-tu...
2,Ngọc Ánh Hotel,"04 Nguyễn Trãi, Thị Trấn Cô Tô, Cô Tô, Quảng N...",20.9689386,107.76418174,https://www.booking.com/hotel/vn/ngoc-anh-quan...
3,Khách sạn Quốc tế Lợi Lai - Li Lai Internation...,"Đường Tuệ Tĩnh, phường Ka Long, thành phố Móng...",21.527927,107.963071,https://www.booking.com/hotel/vn/khach-san-quo...
4,Làng Nương Yên Tử (Yên Tử Village),"Xã Thượng Yên Công, Uông Bí, Việt Nam",21.13477642,106.72447552,https://www.booking.com/hotel/vn/lang-nuong-ye...
...,...,...,...,...,...
988,The Light Hostel,"108 Vườn Đào, Hạ Long, Việt Nam",20.96063778,107.05035271,https://www.booking.com/hotel/vn/the-light-hos...
989,Du thuyền 3 sao - Thăm vịnh Hạ Long DAILY - Bể...,"Tuần Châu, Hạ Long, Việt Nam",20.92248373,106.98306188,https://www.booking.com/hotel/vn/du-thuyen-3-s...
990,Du thuyền ngủ đêm 5 sao trên vịnh Hạ Long - Đẳ...,"Tuần Châu, Hạ Long, Việt Nam",20.91860843,106.98549645,https://www.booking.com/hotel/vn/du-thuyen-5-s...
991,KL HOTEL MARINA SQUARE,"MS 71-72, Ha Long Marina, Ha Long Marine avenu...",20.95703259,107.00151618,https://www.booking.com/hotel/vn/kl-marina-squ...


In [57]:
def normalize(text):
    return unidecode(str(text)).lower() if pd.notnull(text) else None
hotel_data['Normalize'] = hotel_data['Address'].apply(normalize)
normalized_dict = {row['Normalize']: row for _, row in hotel_data.iterrows()}
matched_data = []
for _, row in mongo_df.iterrows():
    housenumber = normalize(row['housenumber_searchName'])
    street = normalize(row['street_searchName'])
    admin_level_4 = normalize(row['admin_level_4_searchName'])
    if None in [housenumber, street, admin_level_4]:
        continue
    for normalized_address, address_row in normalized_dict.items():
        if all(key in normalized_address for key in [housenumber, street, admin_level_4]):
            matched_data.append({
                "_id": row['_id'],
                "name": row['name'],
                "housenumber_searchName": row['housenumber_searchName'],
                "street_searchName": row['street_searchName'],
                "admin_level_4_searchName": row['admin_level_4_searchName'],
                "lat": row['lat'],
                "long": row['lng'],
                "Name": address_row['Hotel Name'],
                "Address": address_row['Address'],
                "Latitude": address_row['Latitude'],
                "Longitude": address_row['Longitude']
            })
matched_df = pd.DataFrame(matched_data)
display(matched_df)


,_id,name,housenumber_searchName,street_searchName,admin_level_4_searchName,lat,long,Name,Address,Latitude,Longitude
0,5ed97a6953921071b5c6c4b9,Wyndham Legend Halong,12,ha long,bai chay,20.959466,107.060374,"Hotel Soleil Ha Long, Trademark Collection by ...","82 Hạ Long 12A Anh Dao, Bai Chay, Ha Long, Qua...",20.956545,107.051103
1,5ed97a6953921071b5c6c4b9,Wyndham Legend Halong,12,ha long,bai chay,20.959466,107.060374,Wyndham Legend Halong,"No 12 Halong Road, Bai Chay ward, Ha Long City...",20.95967629,107.0601663
2,5ed97a6953921071b5c6c4b9,Wyndham Legend Halong,12,ha long,bai chay,20.959466,107.060374,Homestay ECO MAY5 TRAVEL,"20/128 Bãi Cháy, Hạ Long, Việt Nam",20.95570808,107.04328043
3,5ed97a6953921071b5c6c4b9,Wyndham Legend Halong,12,ha long,bai chay,20.959466,107.060374,The One Ha Long Hotel,"N12-1, Khu đô thị mới Cái Dăm, P. Bãi Cháy, TP...",20.9479487,107.02643083
4,5ed97a6953921071b5c6c4b9,Wyndham Legend Halong,12,ha long,bai chay,20.959466,107.060374,Lavian Hotel,"Sun Premier Village, Bai chay, Halong LV3-12, ...",20.944086,107.031298
...,...,...,...,...,...,...,...,...,...,...,...
5554,646b558ed46bae1e2d250ce2,YAKINIKU ĐỒ NƯỚNG KIỂU NHẬT,3,tran phu,tran phu,21.530317,107.967868,LIBRA HOTEL,"136 Triều Dương phường trần phú, Móng Cái, Viê...",21.529445,107.972917
5555,646b558ed46bae1e2d250ce3,"Chuyên Phục Vụ Cho Cá Nhân: Giặt Là, Cắt Tóc, ...",5,tran phu,tran phu,21.530276,107.967805,Majestic Mong Cai Hotel,"No. 5, Hoa Binh Avenue, Tran Phu Ward, Mong Ca...",21.53280984,107.97160506
5556,646b5590d46bae1e2d250e81,Công Ty Tnhh Xnk Thương Đô,36,trieu duong,tran phu,21.532973,107.969685,LIBRA HOTEL,"136 Triều Dương phường trần phú, Móng Cái, Viê...",21.529445,107.972917
5557,64f6ef062b6bd13295ef17d7,Ngân Hàng TMCP Quân Đội CN Móng Cái,2,hung vuong,hoa lac,21.529124,107.968716,Central Hotel,"2 Hùng Vương, Phường Hòa Lạc, Thành phố Móng c...",21.5297271,107.9688262


In [59]:
matched_df.to_csv(output_csv_data, mode='a', index=False, header=False, encoding='utf-8')

In [58]:
def haversine(lat1, lng1, lat2, lng2):
    R = 6371.0 
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lng2 - lng1)
    a = math.sin(delta_phi / 2)**2 + math.cos(phi1) * math.cos(phi2) * math.sin(delta_lambda / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance_km = R * c
    distance_m = distance_km * 1000
    return round(distance_m, 2)
if not matched_df.empty:
    matched_df['Distance'] = matched_df.apply(
        lambda row: haversine(row['lat'], row['long'], row['Latitude'], row['Longitude']), 
        axis=1
    )
    matched_df.to_csv(output_csv_data, mode='a', index=False, header=False, encoding='utf-8')
else:
    print("matched_df is empty, no data to process.")


TypeError: must be real number, not str